In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy

In [ ]:
#carico il modello per l'italiano
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg")

In [ ]:
#imposto il modello BERT da usare con la libreria pipeline
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

In [ ]:
#importo le frasi del dataset dal file drive
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/EXPLICIT_results.xlsx')
sentences = file['sentence_masked']

In [ ]:
#creo due liste per gli output e i prediction scores
output = []
scores = []
for sentence in sentences:
    noun_predictions = []
    #uso l'unmasker per ottenere i primi 10 risultati
    results = unmasker(sentence, top_k=10)
    #per ogni risultato, applico il pos tagging di SpaCy in modo da isolare solo i nomi
    for prediction in results:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                #aggiungo i nomi candidati alla lista noun_predictions
                noun_predictions.append(prediction)
    if noun_predictions:
        #ordino i candidati in base al prediction score
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        #isolo il candidato con lo score più alto
        output.append(noun_predictions[0]["token_str"])
        scores.append(round(noun_predictions[0]["score"], 2))
    else:
        output.append("unknown")
        scores.append("unknown")
#stampo output e punteggi
print(output)
print(scores)